Question 1 : You are working on machine learning project where you have containing numerical and categorical features. You have identified that some features are highly correlated and there are missing values in some of the columns. You want to build a pipeline that automates feature engineering process and handles missing values.
Design a Pipeline that includes following steps:
Use an automated method to identify important features of dataset
Use a numerical pipeline that includes following steps:
Impute missing values in numeric columns with mean
Scale the numerical columns using standardisation
Create a categorical pipeline that includes following steps:
Impute missing values in categorical columns with most frequent data
One Hot Encode the categorical columns
Combine the numerical and categorical pipelines using a ColumnTransformer
Use Random Forest Classifier to build final model
Evaluate accuracy of model on the test dataset
Note : Your Solution should include code snippets for each step of pipeline and a brief explaination of each step. You should also provide an interpretation of results and suggest possible improvements for pipeline

The dataset has the following attributes :

gender - Gender refers to the biological sex of the individual, which can have an impact on their susceptibility to diabetes. There are three categories in it male ,female and other.

age - Age is an important factor as diabetes is more commonly diagnosed in older adults.Age ranges from 0-80 in our dataset.

hypertension - Hypertension is a medical condition in which the blood pressure in the arteries is persistently elevated. It has values a 0 or 1 where 0 indicates they don’t have hypertension and for 1 it means they have hypertension.

heart_disease - Heart disease is another medical condition that is associated with an increased risk of developing diabetes. It has values a 0 or 1 where 0 indicates they don’t have heart disease and for 1 it means they have heart disease.

smoking_history - Smoking history is also considered a risk factor for diabetes and can exacerbate the complications associated with diabetes.In our dataset we have 5 categories i.e not current,former,No Info,current,never and ever.

bmi - BMI (Body Mass Index) is a measure of body fat based on weight and height. Higher BMI values are linked to a higher risk of diabetes. The range of BMI in the dataset is from 10.16 to 71.55. BMI less than 18.5 is underweight, 18.5-24.9 is normal, 25-29.9 is overweight, and 30 or more is obese.

HbA1c_level - HbA1c (Hemoglobin A1c) level is a measure of a person's average blood sugar level over the past 2-3 months. Higher levels indicate a greater risk of developing diabetes. Mostly more than 6.5% of HbA1c Level indicates diabetes.

blood_glucose_level - Blood glucose level refers to the amount of glucose in the bloodstream at a given time. High blood glucose levels are a key indicator of diabetes.

diabetes - Diabetes is the target variable being predicted, with values of 1 indicating the presence of diabetes and 0 indicating the

Dataset used : https://www.kaggle.com/datasets/iammustafatz/diabetes-prediction-dataset

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from warnings import filterwarnings
filterwarnings("ignore")

from sklearn.feature_selection import SelectKBest, mutual_info_classif #For feature selection
from sklearn.impute import SimpleImputer #Handling Missing Values
from sklearn.preprocessing import OneHotEncoder,StandardScaler #Handling Categorical Features & Feature Scaling
from sklearn.pipeline import Pipeline #For automating the entire process
from sklearn.compose import ColumnTransformer #Combining the numerical & Categorial columns
from sklearn.ensemble import RandomForestClassifier #Model for our output predicition

In [2]:
df = pd.read_csv("diabetes_prediction.csv")
df.head()

,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level,diabetes
0,Female,80.0,0,1,never,25.19,6.6,140,0
1,Female,54.0,0,0,No Info,27.32,6.6,80,0
2,Male,28.0,0,0,never,27.32,5.7,158,0
3,Female,36.0,0,0,current,23.45,5.0,155,0
4,Male,76.0,1,1,current,20.14,4.8,155,0


In [3]:
df.shape

(3943, 9)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3943 entries, 0 to 3942
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   gender               3943 non-null   object 
 1   age                  3943 non-null   float64
 2   hypertension         3943 non-null   int64  
 3   heart_disease        3943 non-null   int64  
 4   smoking_history      3943 non-null   object 
 5   bmi                  3943 non-null   float64
 6   HbA1c_level          3943 non-null   float64
 7   blood_glucose_level  3943 non-null   int64  
 8   diabetes             3943 non-null   int64  
dtypes: float64(3), int64(4), object(2)
memory usage: 277.4+ KB


In [5]:
df.isnull().sum()

gender                 0
age                    0
hypertension           0
heart_disease          0
smoking_history        0
bmi                    0
HbA1c_level            0
blood_glucose_level    0
diabetes               0
dtype: int64

In [6]:
#Now seperate dependent and independent variables
X = df.drop('diabetes', axis=1)
y = df['diabetes']

In [7]:
X.shape,y.shape

((3943, 8), (3943,))

In [8]:
cat_cols = list(X.select_dtypes(include='object').columns)
num_cols = list(X.select_dtypes(exclude='object').columns)

In [9]:
#Displaying the list of categorical columns in our dataset
cat_cols

['gender', 'smoking_history']

In [10]:
#Displaying the list of numerical columns in our dataset
num_cols

['age',
 'hypertension',
 'heart_disease',
 'bmi',
 'HbA1c_level',
 'blood_glucose_level']

In [11]:
#Imputing missing values in numerical columns & feature scaling (ie.Standardizing) the values
num_pipeline = Pipeline(
    steps=[
        ('imputer',SimpleImputer(strategy="mean")),
        ('scaler',StandardScaler())
         ]
)

#Imputing missing values in categorical columns & feature Scaling (ie.OneHotEncoding) the values
cat_pipeline = Pipeline(
    steps=[
        ('imputer',SimpleImputer(strategy="most_frequent")),
        ('scaler',OneHotEncoder())
         ]
)

In [12]:
#Combining numerical and categorical pipeline
preprocessor = ColumnTransformer([
    ('num_pipeline',num_pipeline,num_cols),
    ('cat_pipeline',cat_pipeline,cat_cols)
])

In [13]:
# Creating automated pipeline for model selection along with feature selection
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('feature_selector', SelectKBest(score_func=mutual_info_classif, k=7)), #Selecting best 7 features
    ('classifier', RandomForestClassifier())
])

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

X_train.shape , X_test.shape

((2957, 8), (986, 8))

In [15]:
y_train.shape , y_test.shape

((2957,), (986,))

In [16]:
pipeline

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num_pipeline',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['age', 'hypertension',
                                                   'heart_disease', 'bmi',
                                                   'HbA1c_level',
                                                   'blood_glucose_level']),
                                                 ('cat_pipeline',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('scaler',
                                                                   OneHotEncoder())]),
                                                  ['gender',
                                                   'smoking_history'])])),
                ('feature_selector',
                 SelectKBest(k=7,
                             score_func=<function mutual_info_classif at 0x7fbd624417e0>)),
                ('classifier', RandomForestClassifier())])

In [17]:
# Fit the pipeline on the training data
pipeline.fit(X_train, y_train)

# Make predictions on the test data
y_pred = pipeline.predict(X_test)

# Evaluate the accuracy of the model
print(f"Accuracy: {accuracy_score(y_test, y_pred)}")

Accuracy: 0.9746450304259635


Q2. Build a pipeline that includes random forest classifier and a logistic regression classifier , and then voting classifier to combine their predictions. Train the pipeline on iris dataset and evaluate its accuracy

In [18]:
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import VotingClassifier

In [19]:
X,y = load_iris(return_X_y=True)

In [20]:
X.shape , y.shape

((150, 4), (150,))

In [21]:
rfc = RandomForestClassifier()
lr = LogisticRegression()

In [22]:
vc = VotingClassifier(estimators=[('random_forest_classifier',rfc),
                                  ('logistic_regression',lr)],
                                  voting='soft')

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [24]:
X_train.shape , X_test.shape

((112, 4), (38, 4))

In [25]:
y_train.shape , y_test.shape

((112,), (38,))

In [26]:
vc

VotingClassifier(estimators=[('random_forest_classifier',
                              RandomForestClassifier()),
                             ('logistic_regression', LogisticRegression())],
                 voting='soft')

In [27]:
# Fit the Voting Classifier on the training data 
vc.fit(X_train, y_train)

# Make predictions on the test data
y_pred = vc.predict(X_test)

# Evaluate the accuracy of the model
print(f"Accuracy: {accuracy_score(y_test, y_pred)}")

Accuracy: 1.0
